<h1>Web Scraper</h1>
<p>This tool is handy for scraping a website that requires credentials. The credentials have been scrubbed and the downloaded files remain on my local repository for privacy reasons. However, please peruse this code to learn about the neat things you can do with scraping for a particular tag using BeatifulSoup, reading PDF's and tabulating the text into data using PyPDF2 and tabula libraries, and parsing/analyzing/visualizing that tabulated data!</p>

<h2>Scrape them...

In [ ]:
# Import libraries
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

In [ ]:
# Set the URL you want to webscrape from
url = 'https://www.speea.org/Member_Tools/salary_charts/2019/prof_charts.php'

# credentials
payload = {"access_login":"name@company.com","access_password":"password"}

with requests.Session() as session:
    post = session.post(url, data=payload)
    r = session.get(url)
    #print(r.text)   #or whatever else you want to do with the request data!

# Parse HTML and save to BeautifulSoup object¶
soup = BeautifulSoup(r.text, "html.parser")


# To download the whole data set, let's do a for loop through all a tags
for i in range(63,len(soup.findAll('a'))): #'a' tags are for links
    one_a_tag = soup.findAll('a')[i]
    link = one_a_tag['href']
    download_url = 'https://www.speea.org/Member_Tools/salary_charts/2019/'+ link
    urllib.request.urlretrieve(download_url,link[5:-4]+'.pdf') 
    time.sleep(1) #pause the code for a sec

<h2>Time to read them...

In [ ]:
import tabula
import PyPDF2
import numpy as np
import pandas as pd
import os

In [ ]:
col_list = ['Title','Levels', 'Head\rCount', 'Avg\rAge', 'Avg\rSvc', 'Avg\rSalary',
       'After Compa-\rRatio', 'Avg $\rIncr', 'Avg %\rIncr', 'Lower\r10%',
       'Median', 'Upper\r10%', 'Min Mkt', 'Mkt Ref', 'High Mkt', 'Skill Code']
L3df = pd.DataFrame(columns=col_list)

for filename in os.listdir(os.getcwd()+'\\Files'):
    df = tabula.read_pdf(os.getcwd()+'\\Files'+'\\'+filename,pages=1,area=[60,0,100,100],relative_area=True)
    df.rename(columns={'Boeing 2019 Salary Ref. Tbl.\rMinMkt RefHigh Mkt':'Min Mkt','Unnamed: 12':'Mkt Ref','Unnamed: 13':'High Mkt'},inplace=True)
    df['Skill Code'] = filename[:-4]
    pdf_file = open(os.getcwd()+'\\Files'+'\\'+filename,'rb')
    pdf_read = PyPDF2.PdfFileReader(pdf_file)
    page = pdf_read.getPage(0)
    parag = page.extractText()
    title = [line for line in parag.split('\n') if "Salary Distribution by Level for" in line]
    title = title[0][33:]
    df['Title'] = title
    L3df = L3df.append(df[df['Levels']=='Level 2/B'],sort='False',ignore_index=True)[df.columns.tolist()]
    L3df['Skill Code'][L3df.index[-1]] = filename[:-4]
    cols = L3df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    L3df = L3df[cols]

In [ ]:
pd.set_option('display.max_colwidth', 80)
L3df[['Title','Avg\rAge','Avg\rSalary','Lower\r10%','Median','Upper\r10%']]

In [ ]:
L3df_update = L3df[L3df['Avg\rAge']!='- - -']

pd.options.mode.chained_assignment = None  # default='warn'
L3df_update['Avg\rSalary'] = L3df_update['Avg\rSalary'].str.replace('$','').str.replace(',','')
L3df_update['Lower\r10%'] = L3df_update['Lower\r10%'].str.replace('$','').str.replace(',','')

L3df_update['Avg\rSalary'] = L3df_update['Avg\rSalary'].astype(int)
L3df_update['Avg\rAge'] = L3df_update['Avg\rAge'].astype(float)
L3df_update['Head\rCount'] = L3df_update['Head\rCount'].astype(int)
L3df_update['Lower\r10%'] = L3df_update['Lower\r10%'].astype(int)
L3df_update['AgeSal'] = L3df_update['Avg\rSalary']/L3df_update['Avg\rAge']

<h2>Plot them...

In [ ]:
import matplotlib.pyplot as plt

<h4>Set how you want to sort the data, and what Top x skill codes you want to plot</h4>
<p>Only need to sort the data each time you switch the criteria. You can change the topx variable whenever you want.

In [ ]:
L3df_update = L3df_update.sort_values(by=['Lower\r10%'],ascending=False)
L3df_update = L3df_update.reset_index()

In [ ]:
topx = 20 #top x of sorted by

<h4>Use the number variable below to query which skill code you are looking at.

In [ ]:
number = 84
Skill_Code = L3df_update['Skill Code'][L3df_update['index'] == number].iloc[0]
Title_Code = L3df_update['Title'][L3df_update['index'] == number].iloc[0]
print('The Skill Code is: '+Skill_Code)
print('The title is: '+Title_Code)

In [ ]:
sc = L3df_update['Title'][:topx].tolist()
indx = L3df_update['index'][:topx]
fig, ax = plt.subplots(figsize=(14,10))
x = L3df_update['Avg\rAge'][:topx]
yt = L3df_update['Avg\rSalary'][:topx]
y = yt/x
ax.scatter(x,y,s=L3df_update['Head\rCount'][:topx],c=np.random.randint(0, 20, size=min(topx,len(L3df_update))))
#ax.legend(L3df_update['Skill Code'])
for i, txt in enumerate(sc):
    ax.annotate(txt, (x[i], y[i]))


In [ ]:
sc

In [ ]:
l10 = L3df_update['Lower\r10%'][:topx].astype(int)
toplist = pd.DataFrame(list(zip(sc,y,x,yt,l10)),columns=['Title','Salary/Age','Age','Salary','Lower 10%']).sort_values(by=['Lower 10%'],ascending=False)

In [ ]:
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', 500)
toplist

In [ ]:
# Make some labels.
labels = L3df_update['Avg\rAge'][:topx]
x = np.arange(len(labels))  # the label locations
y = L3df_update['Avg\rSalary'][:topx] #salary
fig, ax = plt.subplots(figsize=(14,10))
rects = ax.bar(x, y, .5)


ax.set_title('Avg Salary vs Skill Code with Avg Age')
ax.set_xlabel('Title')
ax.set_ylabel('Avg Salary')
ax.set_xticks(x)
ax.set_xticklabels(L3df_update['Title'][:topx])
plt.xticks(rotation=90)
plt.ylim(min(y)-1000,max(y)+1000)

rects = ax.patches

for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width() / 2, height + 5, label,
            ha='center', va='bottom')

In [ ]:
L3df_update.columns